<a href="https://colab.research.google.com/github/ChaitaliV/Objective-criterias-to-quantify-the-accuracy-of-explanation/blob/main/Explainability%20models/IG_MuRIL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 95.3 MB/s eta 0:00:00


In [3]:
!pip install alibi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 522.7/522.7 KB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 10.7 MB/s eta 0:00:00


In [4]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import torch
from transformers import BertTokenizer
import matplotlib.pyplot as plt
from transformers import pipeline
from alibi.explainers import IntegratedGradients

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [11]:
model=torch.load(r'/content/drive/MyDrive/Saved Models/MuRIL.pt',map_location=torch.device('cpu'))

In [12]:
tokenizer = BertTokenizer.from_pretrained('google/muril-base-cased', do_lower_case=True)

In [13]:
def adapter(data):
  #inputs = tokenizer(data, return_tensors="pt")
  inputs = tokenizer(data, 
          return_tensors='np', 
          padding=True, 
          truncation=True, 
          return_token_type_ids = True,
          return_attention_mask = True,
          max_length=128)
  return inputs

In [14]:
## Define Wrpper for making prediction using MuRIL model
class AutoModelWrapper(tf.keras.Model):

  def __init__(self, model_muril, **kwargs):
    super().__init__()
    self.model_muril = model_muril
  
  def call(self, inputs, attention_mask = None):
    out = self.model_muril(inputs,attention_mask = attention_mask)
    return tf.nn.softmax(out.logits)

  def get_config(self):
    return {}

  @classmethod
  def from_config(cls, config):
    return cls(**config)

auto_model = AutoModelWrapper(model)
max_features = 20000
max_len = 20

In [15]:
# Text sentence for the analysis
z_test_sample = ['I feel a little sad and angry these days']

# Tokenize the sentence and convert it into input tensor
z_test_sample = adapter(z_test_sample)
z_test_sample_input_ids = z_test_sample['input_ids']

# get tensor for model prediction
kwargs = {k: tf.constant(v) for k,v in z_test_sample.items() if k == 'attention_mask'}


In [17]:
## define the integrated gradient method
n_steps = 20
method = 'gausslegendre'
internal_batch_size = 5
ig = IntegratedGradients(auto_model, n_steps = n_steps, method = method, internal_batch_size = internal_batch_size) # TensorFlow model

In [19]:
predictions = auto_model(z_test_sample, **kwargs).numpy().argmax(axis=1)
explanation = ig.explain(z_test_sample, forward_kwargs=kwargs, baseline=None,target=predictions)

RuntimeError: ignored